# Experiments in connecting FAQ with legislative clauses

In [1]:
import numpy as np
import pandas as pd

from src.model import GoldenRetriever
from src.utils import read_txt, split_txt, make_pred, make_iscorr, make_closewrong, make_finetune, make_contrastive_finetune

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def read_and_condition_csv(csv_path):
    """Only read organization meta, not personal. index=196"""
    df_pdpa = pd.read_csv(csv_path)
    df_pdpa = df_pdpa.iloc[:196]
    df_pdpa['kb'] = df_pdpa['meta']+df_pdpa['answer']
    df_pdpa.rename(columns={'question':'queries', 'answer':'answer_str'}, inplace=True)
    df_pdpa['answer'] = [[x] for x in df_pdpa.index]
    df_pdpa['kb']=df_pdpa['kb'].str.replace('\n', '. ').replace('.. ', '. ')
    return df_pdpa
df_pdpa = read_and_condition_csv('./data/pdpa.csv')
print(df_pdpa.columns, len(df_pdpa))
display(df_pdpa.head(3))

Index(['queries', 'meta', 'answer_str', 'kb', 'answer'], dtype='object') 196


,queries,meta,answer_str,kb,answer
0,What is personal data?,"Organisations, General\n","Personal data refers to data, whether true or ...","Organisations, General. Personal data refers t...",[0]
1,When did the PDPA come into force?,"Organisations, General\n",The PDPA was implemented in phases to allow ti...,"Organisations, General. The PDPA was implement...",[1]
2,What are the objectives of the PDPA?,"Organisations, General\n",The PDPA aims to safeguard individuals’ person...,"Organisations, General. The PDPA aims to safeg...",[2]


In [3]:
# Get train/test split index
ind = np.arange(len(df_pdpa))
np.random.seed(0)
np.random.shuffle(ind)
train_ind=ind[:69]
test_ind=ind[69:]

In [4]:
# Initialize model
gr=GoldenRetriever(loss='triplet', margin=0.3)

Instructions for updating:
Colocations handled automatically by placer.


W1023 08:35:55.777341 139924738516736 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1023 08:36:10.507538 139924738516736 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Instructions for updating:
Use tf.cast instead.


W1023 08:36:25.789748 139924738516736 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/losses/losses_impl.py:331: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


W1023 08:36:25.929321 139924738516736 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


model initiated!


## Load KB and make initial test

In [5]:
gr.load_kb(text_list=list(df_pdpa['kb']), question_list=list(df_pdpa['queries']))
df_pdpa['predictions']=df_pdpa.apply(lambda x: make_pred(x, gr), axis=1)
df_pdpa['iscorr']=df_pdpa.apply(lambda x: make_iscorr(x), axis=1)
df_pdpa.iloc[test_ind]['iscorr'].sum()/len(df_pdpa.iloc[test_ind])

knowledge base lock and loaded!


0.6377952755905512

## Finetune model

In [6]:
for epoch in range(40):
    df_pdpa['closewrong']=df_pdpa.apply(lambda x: make_closewrong(x), axis=1)
    gr.finetune(df_pdpa.iloc[train_ind]['queries'], df_pdpa.iloc[train_ind]['kb'], df_pdpa.iloc[train_ind]['kb'], list(df_pdpa.iloc[df_pdpa['closewrong']]['kb'].iloc[train_ind]), list(df_pdpa.iloc[df_pdpa['closewrong']]['kb'].iloc[train_ind]))
#     df_pdpa.iloc[train_ind].apply(lambda x: make_finetune(x, gr), axis=1)
    if epoch%5==0:
        gr.load_kb(text_list=list(df_pdpa['kb']), question_list=list(df_pdpa['queries']))
        df_pdpa['predictions']=df_pdpa.apply(lambda x: make_pred(x, gr), axis=1)
        df_pdpa['iscorr']=df_pdpa.apply(lambda x: make_iscorr(x, 'predictions'), axis=1)
        print(df_pdpa.iloc[test_ind]['iscorr'].sum()/len(df_pdpa.iloc[test_ind]))
        print(df_pdpa.iloc[train_ind]['iscorr'].sum()/len(df_pdpa.iloc[train_ind]))

knowledge base lock and loaded!
0.6456692913385826
0.7536231884057971
knowledge base lock and loaded!
0.6377952755905512
0.7681159420289855
knowledge base lock and loaded!
0.6535433070866141
0.7971014492753623
knowledge base lock and loaded!
0.6456692913385826
0.8115942028985508
knowledge base lock and loaded!
0.6614173228346457
0.8260869565217391
knowledge base lock and loaded!
0.6692913385826772
0.8840579710144928
knowledge base lock and loaded!
0.6692913385826772
0.8985507246376812
knowledge base lock and loaded!
0.6850393700787402
0.8985507246376812


In [7]:
df_pdpa_legis = pd.read_csv('./data/pdpa_legis.csv')
df_pdpa_legis['kb'] = df_pdpa_legis['meta']+df_pdpa_legis['answer']

In [10]:
gr.load_kb(text_list=list(df_pdpa_legis['kb']))
df_pdpa['predictions_legis']=df_pdpa.apply(lambda x: make_pred(x, gr), axis=1)

knowledge base lock and loaded!


In [ ]:
def display_ans(row):
    print('QN: ',row['queries'], 'CORRECT: ',row['answer_str'])
#     for pred in row['predictions']:
    for item in list(df_pdpa['kb'].iloc[row['predictions']]):
        print('FAQ: ',item)
    for item in list(df_pdpa_legis['kb'].iloc[row['predictions_legis']]):
        print('LEGIS: ', item)
    print('\n')
    
df_pdpa.apply(lambda x: display_ans(x), axis=1)

QN:  What is personal data? CORRECT:  Personal data refers to data, whether true or not, about an individual who can be identified from that data; or from that data and other information to which the organisation has or is likely to have access.This includes unique identifiers (e.g. NRIC number, passport number); photographs or video images of an individual (e.g. CCTV images); as well as any set of data (e.g. name, age, address, telephone number, occupation, etc), which when taken together would be able to identify the individual. For example, Jack Lim, 36 years old, civil servant, lives at Blk 123 Bishan St 23.
FAQ:  Organisations, Care of Personal Data. The PDPA does not prescribe the retention period of personal data. However, an organisation should cease to retain its documents containing personal data, or remove the means by which the personal data can be associated with particular individuals, as soon as it is reasonable to assume that the purpose of collection is no longer serve